This notebook takes code from 'AZoneDissolve_AB.ipynb' and runs the entire state of texas

In [1]:
import os
import glob
import math
from processing_tools import *
from shapely.geometry import MultiPoint
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.ops import triangulate


In [9]:
""" Read and collect AZones for each TX county into single dataframe, then dissolve """
gdf_list = []
state_path = 'D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\TEST_CountyLevel_Processing'
for fi in glob.glob(os.path.join(state_path, '08013_FZ.shp')):
    print('Reading', fi)
    fz = gpd.read_file(fi)
    gdf_list.append(fz.loc[fz['fz_type'] == 'A'])
full_gdf = pd.concat(gdf_list, ignore_index=True)
print('Dissolving....')
dis = full_gdf.dissolve()



Reading D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\TEST_CountyLevel_Processing\08013_FZ.shp
Dissolving....


In [10]:
# determine correct utm code
crs = utm_code(dis)
crs

'32613'

In [12]:
dis = dis.explode()
dis.shape

(28, 2)

In [13]:
elevVrt = r'D:\GitHub_Repos\GIS\BFEs\Workspace\WorkingScripts_v1014_2022\Preprocessing\OtherZone_testing\08013_1m.vrt'


In [8]:
pt_geom

In [15]:
""" Using only vertices for triangulation """
triangles = gpd.GeoDataFrame()
for i, d in dis.iterrows():
    print(i)
    d84 = g(d.geometry, crs)

    # Vertice coordinates
    coords = list(d.geometry.exterior.coords)

    # list of shapely objects
    pt_geom = [Point(c) for c in coords]

    # Extract elev from DEM
    elev = GetElevation(pt_geom, elevVrt)

    # Convert to MultiPoint object
    mpts = MultiPoint(elev)

    # Triangulate
    tin = triangulate(mpts)

    # Convert to geodataframe
    tin = g(tin, 4326)

    # Extract geom to attrib table
    final_tin = extract_geom(tin)

    # Clip to triangles within zone
    final_tin = final_tin.overlay(d84)
                        

    # Collect
    triangles = pd.concat([triangles, final_tin], ignore_index=True)
    


(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)


In [16]:
triangles.to_file('08013_Azone_tri_clip.shp')